
---

## **Atividade 1 — Classificação Supervisionada**

Nesta atividade, você deve **comparar quatro diferentes paradigmas de aprendizado de máquina**: probabilísticos, simbólicos, conexionistas e estatísticos.

> **Importante:** Durante a **etapa de inferência**, o modelo **só poderá utilizar o conteúdo textual** (`content`) do post para prever o engajamento. Informações como `reactions` e `comments` devem ser utilizadas apenas na etapa de treinamento, se desejar, mas não podem ser usadas como entrada no momento da predição de novos posts (eles não estarão disponíveis em inferência).

Para avaliação, será disponibilizado um arquivo de teste contendo **novas postagens sem o rótulo de engajamento**, e cada aluno deverá submeter suas **previsões finais de engajamento** para essas novas entradas.

In [ ]:
!gdown 1csTzz_UWpe89KYh11qj_V7fDssTv2FFO
!gdown 1KlZGlcUEVgVVvxV0l4vYwQ1g1pgmsYio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
#from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


In [ ]:
df = pd.read_csv('df_social_data_train.csv')
df_teste = pd.read_csv('df_social_data_test.csv')
df

In [ ]:
df_teste

### Um pequeno pré-processamento dos dados para utilizarmos com os modelos que virão

In [ ]:
# pre-processar e extrair caracteristicas dos dados para construir a tabela atributo-valor
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
df = df.dropna()
df['features'] = list(model.encode(df['content'].tolist(), show_progress_bar=True))
df

In [ ]:
# Trocamos o label para '0' (low) ou '1' (high) --> Bibliotecas trabalham melhor com labels numéricas
df['engagement'] = df['engagement'].map({'low': 0, 'high': 1})

In [ ]:
# Separação dos dados e labels
labels = df['engagement'].to_numpy()
data = np.array(df['features'].tolist())

Criação de uma função KFold para melhor validação dos resultados de classificação
- Alguns modelos demoram muito para processar, por isso o parâmetro 'k' é baixo em alguns testes

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

'''
    data --> features
    label --> 0 (low) ou 1 (high)
    k --> número de iterações do KFold
    classifier --> modelo de classificação que estamos usando
'''
def applyKfold(data, label, k, classifier):

    kf = StratifiedKFold(n_splits=k, random_state=42, shuffle=True)
    accVet = []
    f1Vet = []
    f1MacroVet = []

    # To store all true and predicted labels
    all_y_true = []
    all_y_pred = []

    for train_index, test_index in kf.split(data, label):
        # Split data
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # Train classifier
        clf = classifier
        clf.fit(data_train, label_train)

        # Predict
        y_pred = clf.predict(data_test)

        # Save metrics
        acc = accuracy_score(label_test, y_pred)
        f1 = f1_score(label_test, y_pred, average='binary')
        f1_macro = f1_score(label_test, y_pred, average='macro')

        accVet.append(acc)
        f1Vet.append(f1)
        f1MacroVet.append(f1_macro)

        # Save all predictions for final report
        all_y_true.extend(label_test)
        all_y_pred.extend(y_pred)

    # Final classification report across all folds
    print("\n=== Final Classification Report (All Folds Combined) ===")
    print(classification_report(all_y_true, all_y_pred, digits=3))

    return (accVet, f1Vet, f1MacroVet)



---
## Abordagem **Probabilística** - Naive Bayes

In [ ]:
acc, f1, f1Macro = applyKfold(data, labels, 10, GaussianNB())
print("Accuracy Score: ", round(np.mean(acc), 4))
print("F1 Score: ", round(np.mean(f1), 4))
print("F1 Macro Score: ", round(np.mean(f1Macro), 4))


---
## Abordagem **Simbólica** - Decision Tree

In [ ]:
acc, f1, f1Macro = applyKfold(data, labels, 3, DecisionTreeClassifier(criterion='gini', max_depth=15))
print("Accuracy Score: ", round(np.mean(acc), 4))
print("F1 Score: ", round(np.mean(f1), 4))
print("F1 Macro Score: ", round(np.mean(f1Macro), 4))


---
## Abordagem **Conexionista** - Multilayer Perceptron

Para essa abordagem, duas maneiras diferentes foram utilizadas para testar o Multilayer Perceptron:
- Utilizando o Keras models do Tensorflow
- Utilizando o sklearn com suas neural networks

Para ambos os casos foi utilizado o 'early stopping' para evitar eventuais overfittings

MLP com o Keras

In [ ]:
value_1 = 32
value_2 = 16

mlp = Sequential()
mlp.add(Dense(value_1, input_shape=(data.shape[1],), activation='tanh'))
mlp.add(Dropout(0.1))
mlp.add(Dense(value_2, input_shape=(data.shape[1],), activation='tanh'))
mlp.add(Dropout(0.1))
mlp.add(Dense(1, input_shape=(data.shape[1],), activation='sigmoid'))

mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

history = mlp.fit(
    X_train,
    y_train,
    epochs=1000,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Plotando curva de aprendizado
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title("Curva de Aprendizado")
plt.show()

In [ ]:
# Avaliar no conjunto de teste
test_loss, test_acc = mlp.evaluate(X_test, y_test, verbose=1)

y_pred_proba = mlp.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

print(classification_report(y_test, y_pred, digits=3))
print(f"[MLP] Acurácia no teste final: {test_acc:.4f}, Loss: {test_loss:.4f}")

MLP com o sklearn

In [ ]:
# Avaliar MLP com o sklearn
from sklearn.neural_network import MLPClassifier
acc, f1, f1Macro = applyKfold(data, labels, 2, MLPClassifier(early_stopping=True, max_iter=300))
print("Accuracy Score: ", round(np.mean(acc), 4))
print("F1 Score: ", round(np.mean(f1), 4))
print("F1 Macro Score: ", round(np.mean(f1Macro), 4))


---
## Abordagem **Estatística** - SVC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# classificador
clf_svc = SVC(kernel="rbf", C=1, gamma='scale', max_iter=10000, class_weight='balanced')
clf_svc.fit(X_train_scaled, y_train)

# predição
y_pred = clf_svc.predict(X_test_scaled)

# avaliação
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("Accuracy Score:", round(acc, 4))
print("F1 Score:", round(f1, 4))
print("F1 Macro Score:", round(f1_macro, 4))

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred, digits=3))
print(confusion_matrix(y_test, y_pred))

In [ ]:
df_teste['content'] = df_teste['content'].astype(str)
df_teste['features'] = list(model.encode(df_teste['content'].tolist(), show_progress_bar=True))
X_teste_final = scaler.transform(df_teste['features'].tolist())
df_teste['Engagement'] = clf_svc.predict(X_teste_final)
df_teste['ID'] = range(len(df_teste))
df_teste

In [ ]:
df_teste['Engagement'].value_counts()

In [ ]:
df_teste[['ID','Engagement']].to_csv('df_kaggle.csv', index=False)

### Conclusão

O que apresentou o melhor resultado nesta análise foi o classificador SVC na abordagem estatística